# 🤖 Dual CNN Emotion Inference + BERT Verifier
This notebook loads two CNN-based facial emotion models, compares their predictions, and uses a BERT-based verifier to choose or correct the final result.

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.ngc.nvidia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
     |████████████████████████████████| 10.0 MB 5.2 MB/s eta 0:00:01
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'pypi.ngc.nvidia.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
     |████████████████████████████████| 783 kB 13.5 MB/s eta 0:00:01
/usr/share/python-wheels/urllib3-1.25.8-py2.py3-none-any.whl/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being 

In [7]:
!pip install transformers --no-deps
!pip install numpy tokenizers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [6]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import model_from_json
from transformers import pipeline
from tensorflow.keras.preprocessing.image import img_to_array

import pandas as pd
import shutil

RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
/home/aarons/.local/lib/python3.8/site-packages/sklearn/__check_build/../../scikit_learn.libs/libgomp-d22c30c5.so.1.0.0: cannot allocate memory in static TLS block
___________________________________________________________________________
Contents of /home/aarons/.local/lib/python3.8/site-packages/sklearn/__check_build:
__init__.py               _check_build.cpython-38-aarch64-linux-gnu.so__pycache__
___________________________________________________________________________
It seems that scikit-learn has not been built correctly.

If you have installed scikit-learn from source, please do not forget
to build the package before using it: run `python setup.py install` or
`make` in the source directory.

If you have used an installer, please check that it is suited for your
Python version, your operating system and your platform.

In [ ]:

# === CONFIG ===
image_folder = "datset"  # Folder with test images
image_size = (48, 48)         # Resize for CNNs
EMOTIONS = ["angry", "disgusted", "fearful", "happy", "neutral", "sad", "surprised"]

# Load both models (adjust paths if needed)
def load_model_from_json(json_path, weights_path):
    with open(json_path, "r") as f:
        model = model_from_json(f.read())
    model.load_weights(weights_path)
    return model

model1 = load_model_from_json("models/model1.json", "models/model1_weights.h5")
model2 = load_model_from_json("models/model2.json", "models/model2_weights.h5")


In [ ]:

# === BERT Verifier Setup ===
emotion_verifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion", return_all_scores=True)


In [ ]:

# === Dual Inference + BERT Correction ===
def verify_with_bert(text1, text2):
    prompt = f"Which emotion is more accurate: '{text1}' or '{text2}'?"
    scores = emotion_verifier(prompt)[0]
    ranked = sorted(scores, key=lambda x: x['score'], reverse=True)
    return ranked[0]['label'], ranked[0]['score']

results = []

for img_file in os.listdir(image_folder):
    if not img_file.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    path = os.path.join(image_folder, img_file)
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, image_size)
    face_img = resized.astype("float32") / 255.0
    face_img = img_to_array(face_img)
    face_img = np.expand_dims(face_img, axis=0)

    pred1 = model1.predict(face_img)[0]
    pred2 = model2.predict(face_img)[0]

    label1 = EMOTIONS[np.argmax(pred1)]
    label2 = EMOTIONS[np.argmax(pred2)]

    if label1 == label2:
        final_label = label1
        verified = "✔️ Match"
    else:
        final_label, score = verify_with_bert(label1, label2)
        verified = f"🤖 BERT chose: {final_label} (conf: {score:.2f})"

    print(f"🖼️ {img_file} → M1: {label1}, M2: {label2} → Final: {final_label} | {verified}")
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"{final_label} ({verified})")
    plt.axis('off')
    plt.show()

    results.append({
        "filename": img_file,
        "model1": label1,
        "model2": label2,
        "final": final_label,
        "verified": verified
    })


In [ ]:

# === SAVE VERIFIED LABELS + COPY IMAGES TO pseudo_train/ ===
print("\nSaving verified results and creating pseudo-labeled dataset...")
df = pd.DataFrame(results)
df.to_csv("verified_labels.csv", index=False)

pseudo_dir = "pseudo_train"
os.makedirs(pseudo_dir, exist_ok=True)

for row in results:
    emotion = row['final']
    src_path = os.path.join(image_folder, row['filename'])
    dst_folder = os.path.join(pseudo_dir, emotion)
    os.makedirs(dst_folder, exist_ok=True)
    dst_path = os.path.join(dst_folder, row['filename'])
    shutil.copyfile(src_path, dst_path)

print("✅ Results saved to 'verified_labels.csv'")
print("✅ Images copied into 'pseudo_train/<emotion>/' folders")
